In [1]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import create_engine
import os
import pandas as pd

app = Flask(__name__)

engine = create_engine('sqlite:///recommender2.db', echo=False)

for f in os.listdir('data/movies/ml-latest-small'):
    if f[-4:] == '.csv':
        data = pd.read_csv(f'data/movies/ml-latest-small/{f}')
        data.to_sql(f[:-4], engine)
        print(f[0:-4])

links
ratings
movies
tags


In [2]:
watched_movie_id_list = ['70286', '109487', '589']

In [3]:
import numpy as np

movie_id_unique = 'SELECT * FROM movies'
all_movies = pd.read_sql(movie_id_unique, engine)

#remove the input movies from all_movies
movies_not_watched = all_movies[~all_movies['movieId'].isin(watched_movie_id_list)]
movies_not_watched.loc[:,'fake_id'] = np.ones(len(movies_not_watched), dtype =int)

#get all_ratings from sqlite
query = 'SELECT "userId", ratings."movieId", movies.title, rating FROM ratings JOIN movies ON ratings."movieId" = movies."movieId";'
all_ratings = pd.read_sql(query, engine)

#remove the watched movies from all_ratings
not_all_ratings = all_ratings[~all_ratings['movieId'].isin(watched_movie_id_list)]

#remove movieId and ratings of the watched movies???
movieindex = not_all_ratings['movieId'].unique().tolist()
dl_movie2movie_encoded = {x: i for i, x in enumerate(movieindex)}
dl_movie_encoded2movie = {i: x for i, x in enumerate(movieindex)}

not_all_ratings.loc[:,"movie"] = not_all_ratings["movieId"].map(dl_movie2movie_encoded)
not_all_ratings.loc[:,"rating"] = not_all_ratings["rating"].values.astype(np.float32)

#map userId from ?? to not_all_ratings ?? what do i do with the users??

not_all_user_ids = not_all_ratings["userId"].unique().tolist()
dl_user2user_encoded = {x: i for i, x in enumerate(not_all_user_ids)}
dl_userencoded2user = {i: x for i, x in enumerate(not_all_user_ids)}

not_all_ratings.loc[:,"user"] = not_all_ratings["userId"].map(dl_user2user_encoded)

min_rating = min(not_all_ratings["rating"])
max_rating = max(not_all_ratings["rating"])
num_users = len(dl_user2user_encoded)
num_movies = len(dl_movie_encoded2movie)
print(
        "Number of users: {}, Number of Movies: {}, Min rating: {}, Max rating: {}".format(
            num_users, num_movies, min_rating, max_rating
        )
    )

#define training data
df = not_all_ratings.sample(frac=1, random_state=42)
x = not_all_ratings[["user", "movie"]].values

    # Normalize the targets between 0 and 1. Makes it easy to train.
y = not_all_ratings["rating"].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values
    # Assuming training on 90% of the data and validating on 10%.
train_indices = int(0.9 * df.shape[0])
x_train, x_val, y_train, y_val = (
        x[:train_indices],
        x[train_indices:],
        y[:train_indices],
        y[train_indices:],
    )

#inference
user_movie_array = movies_not_watched[['fake_id','index']]
max_movie_index = user_movie_array['index'].max()
that = user_movie_array.to_numpy()

EMBEDDING_SIZE = 50

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class RecommenderNet(keras.Model):
        def __init__(self, num_users, num_movies, embedding_size, **kwargs):
            super(RecommenderNet, self).__init__(**kwargs)
            self.num_users = num_users
            self.num_movies = num_movies
            self.embedding_size = embedding_size
            self.user_embedding = layers.Embedding(
                num_users,
                embedding_size,
                embeddings_initializer="he_normal",
                embeddings_regularizer=keras.regularizers.l2(1e-6),
            )
            self.user_bias = layers.Embedding(num_users, 1)
            self.movie_embedding = layers.Embedding(
            #change this line from num_movies to max_movie_index+1
                max_movie_index+1,
                embedding_size,
                embeddings_initializer="he_normal",
                embeddings_regularizer=keras.regularizers.l2(1e-6),
            )
            #and this line (input_dim)
            self.movie_bias = layers.Embedding(max_movie_index+1, 1)

        def call(self, inputs):
            user_vector = self.user_embedding(inputs[:, 0])
            user_bias = self.user_bias(inputs[:, 0])
            movie_vector = self.movie_embedding(inputs[:, 1])
            movie_bias = self.movie_bias(inputs[:, 1])
            dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
            # Add all the components (including bias)
            x = dot_user_movie + user_bias + movie_bias
            # The sigmoid activation forces the rating to between 0 and 1
            return tf.nn.sigmoid(x)
            #return movie_bias

model = RecommenderNet(num_users, num_movies, EMBEDDING_SIZE)
model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(lr=0.001)
    )
ratings = model.predict(that).flatten()

#top_ratings_indices = ratings.argsort()[-10:][::-1]

movies_not_watched.loc[:,'prediction'] = ratings
highest_score = ratings[ratings.argsort()[-10:]][::-1]

recom_movie_titles = movies_not_watched.loc[movies_not_watched.loc[:,'prediction'].isin(highest_score)]
#movies_not_watched.loc[movies_not_watched['index'].isin(top_ratings_indices)]
recom_movie_titles

/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


Number of users: 610, Number of Movies: 9721, Min rating: 0.5, Max rating: 5.0


/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


,index,movieId,title,genres,fake_id,prediction
792,792,1035,"Sound of Music, The (1965)",Musical|Romance,1,0.527315
793,793,1036,Die Hard (1988),Action|Crime|Thriller,1,0.527908
2051,2051,2730,Barry Lyndon (1975),Drama|Romance|War,1,0.529678
2061,2061,2740,"Kindred, The (1986)",Horror|Sci-Fi,1,0.528041
6627,6627,56156,Hitman (2007),Action|Crime|Thriller,1,0.528539
6635,6635,56333,"Savages, The (2007)",Comedy|Drama,1,0.529852
7404,7404,80124,Sisters (Syostry) (2001),Action|Crime|Drama,1,0.527251
7407,7407,80162,"Horde, The (La Horde) (2009)",Action|Horror|Thriller,1,0.530841
7412,7412,80454,Princess (Prinsessa) (2010),Drama,1,0.531843
7421,7421,80586,Flipped (2010),Comedy|Drama|Romance,1,0.528281


In [4]:
liked_genres = all_movies.loc[all_movies['movieId'].isin(watched_movie_id_list), 'genres'].to_list()
liked_genres

['Action|Sci-Fi', 'Mystery|Sci-Fi|Thriller', 'Sci-Fi|IMAX']

In [5]:
splitted = [sub.split("|") for sub in liked_genres]


In [6]:
flat_list = []
for sublist in splitted:
    for item in sublist:
        flat_list.append(item)

In [7]:
flat_list

['Action', 'Sci-Fi', 'Mystery', 'Sci-Fi', 'Thriller', 'Sci-Fi', 'IMAX']

In [8]:


all_movies['genres'].replace('Sci-Fi', 'SciFi', inplace=True, regex=True)
all_movies['genres'].replace('Film-Noir', 'FilmNoir', inplace=True, regex=True)
all_movies['genres'][300:350]

300                                             Comedy
301                                           Children
302                                             Comedy
303                                       Comedy|Drama
304                                      Drama|Musical
305                             Drama|FilmNoir|Romance
306                                             Comedy
307                        Action|Crime|Drama|Thriller
308                             Drama|Mystery|Thriller
309                               Comedy|Drama|Romance
310                                       Comedy|Drama
311                      Action|Crime|Fantasy|Thriller
312                                              Drama
313                            Children|Comedy|Fantasy
314                           Comedy|Drama|Romance|War
315                                     Comedy|Romance
316                                              Drama
317                               Comedy|Drama|Romance
318       

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
all_movies['genres'].replace('(no genres listed)', 'N/A', inplace=True, regex=True)


In [12]:
allgenres = all_movies['genres']
tfidf_vectorizer=TfidfVectorizer()
tfidf_matrix=tfidf_vectorizer.fit_transform(allgenres)

print(tfidf_vectorizer.get_feature_names()) #นี่ไง 24 ทนที่จะเป็น 20 อี sci-fi นี่หาเรื่องตลอดเลยนะ

['action', 'adventure', 'animation', 'children', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'filmnoir', 'horror', 'imax', 'musical', 'mystery', 'romance', 'scifi', 'thriller', 'war', 'western']


In [14]:
tfidf_matrix #no genres listed!

<9742x22 sparse matrix of type '<class 'numpy.float64'>'
	with 22152 stored elements in Compressed Sparse Row format>

In [15]:
allgenres[9663:9665]

9663           (no genres listed)
9664    Animation|Children|Comedy
Name: genres, dtype: object

In [11]:
allgenres = all_movies['genres']
tfidf_vectorizer=TfidfVectorizer()
tfidf_matrix=tfidf_vectorizer.fit_transform(allgenres)

print(tfidf_vectorizer.get_feature_names()) 

['action', 'adventure', 'animation', 'children', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'filmnoir', 'horror', 'imax', 'musical', 'mystery', 'romance', 'scifi', 'thriller', 'war', 'western']


In [13]:
len(tfidf_vectorizer.get_feature_names()) 

19

In [13]:
splitted = [sub.split("|") for sub in allgenres]
flat_genres_list = [item for sublist in splitted for item in sublist]
flat_genres_list

['Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Fantasy',
 'Adventure',
 'Children',
 'Fantasy',
 'Comedy',
 'Romance',
 'Comedy',
 'Drama',
 'Romance',
 'Comedy',
 'Action',
 'Crime',
 'Thriller',
 'Comedy',
 'Romance',
 'Adventure',
 'Children',
 'Action',
 'Action',
 'Adventure',
 'Thriller',
 'Comedy',
 'Drama',
 'Romance',
 'Comedy',
 'Horror',
 'Adventure',
 'Animation',
 'Children',
 'Drama',
 'Action',
 'Adventure',
 'Romance',
 'Crime',
 'Drama',
 'Drama',
 'Romance',
 'Comedy',
 'Comedy',
 'Action',
 'Comedy',
 'Crime',
 'Drama',
 'Thriller',
 'Comedy',
 'Crime',
 'Thriller',
 'Crime',
 'Drama',
 'Horror',
 'Mystery',
 'Thriller',
 'Action',
 'Crime',
 'Thriller',
 'Drama',
 'SciFi',
 'Drama',
 'Romance',
 'Drama',
 'Children',
 'Drama',
 'Drama',
 'Romance',
 'Adventure',
 'Drama',
 'Fantasy',
 'Mystery',
 'SciFi',
 'Crime',
 'Drama',
 'Drama',
 'Mystery',
 'SciFi',
 'Thriller',
 'Children',
 'Drama',
 'Crime',
 'Drama',
 'Children',
 'Comedy',
 'Comedy',
 'Romance',
 

In [20]:
len(flat_genres_list)

22084

In [14]:
np.unique(flat_genres_list)

array(['(N/A)', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'FilmNoir', 'Horror',
       'IMAX', 'Musical', 'Mystery', 'Romance', 'SciFi', 'Thriller',
       'War', 'Western'], dtype='<U11')

In [22]:
len(np.unique(flat_genres_list))

20

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectors = vectorizer.fit_transform(allgenres.to_dict().values())
vectors

<9742x19 sparse matrix of type '<class 'numpy.int64'>'
	with 22050 stored elements in Compressed Sparse Row format>

In [69]:
vectorizer.get_feature_names()

['action',
 'adventure',
 'animation',
 'children',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'fantasy',
 'filmnoir',
 'horror',
 'imax',
 'musical',
 'mystery',
 'romance',
 'scifi',
 'thriller',
 'war',
 'western']

In [15]:
pd.DataFrame(vectors.todense())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9738,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9739,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
9740,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
all_movies['genres'][0]

'Adventure|Animation|Children|Comedy|Fantasy'

In [16]:
pd.DataFrame(tfidf_matrix.todense())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.000000,0.416846,0.516225,0.504845,0.267586,0.0,0.0,0.000000,0.482990,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.000000,0.512361,0.000000,0.620525,0.000000,0.0,0.0,0.000000,0.593662,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.570915,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.505015,0.0,0.0,0.466405,0.000000,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,0.436010,0.000000,0.614603,0.000000,0.318581,0.0,0.0,0.000000,0.575034,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9738,0.000000,0.000000,0.682937,0.000000,0.354002,0.0,0.0,0.000000,0.638968,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9739,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9740,0.578606,0.000000,0.815607,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [ ]:
#vectorize each line of genres into tfidf vector, or actually can be just countvector? will conclude that later
#at least now the shape is correct 9742*19

In [ ]:
#then for the liked_genres, do the same thing to remove hyphen from Sci-Fi, Film-Noir, and (no genres listed)

In [27]:
flat_list

['Action', 'Sci-Fi', 'Mystery', 'Sci-Fi', 'Thriller', 'Sci-Fi', 'IMAX']

In [19]:
liked_genres = all_movies.loc[all_movies['movieId'].isin(watched_movie_id_list), 'genres'].to_list()
#take out the hyphen to correct the len of feature_names() --should be 5 genres with 3 times scifi

splitted = [sub.split("|") for sub in liked_genres]
flat_list = []
for sublist in splitted:
    for item in sublist:
        flat_list.append(item)
flat_list

['Action', 'SciFi', 'Mystery', 'SciFi', 'Thriller', 'SciFi', 'IMAX']

In [128]:
liked_genres = all_movies.loc[all_movies['movieId'].isin(watched_movie_id_list), 'genres'].to_list()
liked_genres

['Action|SciFi', 'Mystery|SciFi|Thriller', 'SciFi|IMAX']

In [ ]:
watched_movie_id_list = ['70286', '109487', '589']

In [20]:
target = TfidfVectorizer()
tarrget = target.fit_transform(flat_list)
tarrget

<7x5 sparse matrix of type '<class 'numpy.float64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [21]:
target.get_feature_names()

['action', 'imax', 'mystery', 'scifi', 'thriller']

In [21]:
countlike = CountVectorizer()
countlikevec = countlike.fit_transform(flat_list)
countlikevec

<7x5 sparse matrix of type '<class 'numpy.int64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [22]:
countlike.get_feature_names()

['action', 'imax', 'mystery', 'scifi', 'thriller']

In [42]:
pd.DataFrame(countlikevec.todense())

,0,1,2,3,4
0,1,0,0,0,0
1,0,0,0,1,0
2,0,0,1,0,0
3,0,0,0,1,0
4,0,0,0,0,1
5,0,0,0,1,0
6,0,1,0,0,0


In [44]:
pd.DataFrame(tarrget.todense())

,0,1,2,3,4
0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0
5,0.0,0.0,0.0,1.0,0.0
6,0.0,1.0,0.0,0.0,0.0


In [47]:
tfidf_matrix

<9742x19 sparse matrix of type '<class 'numpy.float64'>'
	with 22050 stored elements in Compressed Sparse Row format>

In [48]:
tarrget

<7x5 sparse matrix of type '<class 'numpy.float64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [ ]:
#have to convert tarrget to be 1*19, can it be 7*19?
#how?
#then can i combine all 7 rows to 1?

In [49]:
from sklearn.metrics.pairwise import cosine_similarity

cs = cosine_similarity(tfidf_matrix,tarrget)
print (cs)

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 19 while Y.shape[1] == 5

In [ ]:
#have to turn 5 to 19 with one hot encoding?

In [52]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
X = [['Male', 1], ['Female', 3], ['Female', 2]]
>>> enc.fit(X)
OneHotEncoder(handle_unknown='ignore')
>>> enc.categories_
[array(['Female', 'Male'], dtype=object), array([1, 2, 3], dtype=object)]
>>> enc.transform([['Female', 1], ['Male', 4]]).toarray()
array([[1., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]])
>>> enc.inverse_transform([[0, 1, 1, 0, 0], [0, 0, 0, 1, 0]])
array([['Male', 1],
       [None, 2]], dtype=object)
>>> enc.get_feature_names(['gender', 'group'])
array(['gender_Female', 'gender_Male', 'group_1', 'group_2', 'group_3'],
  dtype=object)

In [ ]:
 fit_transform(X, y=None)
    X:array-like, shape [n_samples, n_features]

In [55]:
tarrgetdf = pd.DataFrame(tarrget.todense())
X = tarrgetdf.to_numpy()
X

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.]])

In [61]:
X.shape

(7, 5)

In [58]:
enc = OneHotEncoder(handle_unknown='ignore')

tarrgetenc = enc.fit_transform(X, y=None)
tarrgetenc

<7x10 sparse matrix of type '<class 'numpy.float64'>'
	with 35 stored elements in Compressed Sparse Row format>

In [60]:
pd.DataFrame(tarrgetenc.todense()) #what's happening here?

,0,1,2,3,4,5,6,7,8,9
0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
2,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
3,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
5,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
6,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0


In [ ]:
#let's try labelBinarizer instead of one hot encoder

from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
>>> lb.fit([1, 2, 6, 4, 2])
LabelBinarizer()
>>> lb.classes_
array([1, 2, 4, 6])
>>> lb.transform([1, 6])
array([[1, 0, 0, 0],
       [0, 0, 0, 1]])

In [23]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(vectorizer.get_feature_names())



LabelBinarizer()

In [24]:
lb.classes_


array(['action', 'adventure', 'animation', 'children', 'comedy', 'crime',
       'documentary', 'drama', 'fantasy', 'filmnoir', 'horror', 'imax',
       'musical', 'mystery', 'romance', 'scifi', 'thriller', 'war',
       'western'], dtype='<U11')

In [25]:
countlike

CountVectorizer()

In [26]:
lb.transform(countlike.get_feature_names())

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]])

In [ ]:
#เราต้องทำเป็น multidimension สินะ
ต้องทำ lb.transform ของ 3 เรื่อง ให้ออกมาเป็นแบบข้างบน สมมติข้างบนเป็น 1 เรื่อง ไม่ใช่ทั้งหมด
หรือไม่ก็ต้องทำให้ scifi มันหนักกว่าอันอื่น แล้วมันจะเอามา cosine_similarity กันยังไง?

In [67]:
countlike.get_feature_names()

['action', 'imax', 'mystery', 'scifi', 'thriller']

In [75]:
liked_genres

['Action|SciFi', 'Mystery|SciFi|Thriller', 'SciFi|IMAX']

In [27]:
flat_list

['Action', 'SciFi', 'Mystery', 'SciFi', 'Thriller', 'SciFi', 'IMAX']

In [28]:
for item in flat_list:
    item.lower()

In [29]:
lowercase = [item.lower() for item in flat_list]
lowercase #query

['action', 'scifi', 'mystery', 'scifi', 'thriller', 'scifi', 'imax']

In [77]:
lb.transform(flat_list)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [30]:
lb.transform(lowercase)
#เริ่มเข้าเค้า
#ทีนี้จะ vectorize อันนี้ได้ไหม

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

In [35]:
likedbinary = lb.transform(lowercase)

In [84]:
target = TfidfVectorizer()
likedvector = target.fit_transform(likedbinary)
likedvector

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [68]:
np.unique(flat_genres_list)

array(['(N/A)', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'FilmNoir', 'Horror',
       'IMAX', 'Musical', 'Mystery', 'Romance', 'SciFi', 'Thriller',
       'War', 'Western'], dtype='<U11')

In [31]:
vectorizer.get_feature_names() #document ไม่ใช่! document ต้องมี 9742 บรรทัดสิ

['action',
 'adventure',
 'animation',
 'children',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'fantasy',
 'filmnoir',
 'horror',
 'imax',
 'musical',
 'mystery',
 'romance',
 'scifi',
 'thriller',
 'war',
 'western']

In [32]:
lowercase #query

['action', 'scifi', 'mystery', 'scifi', 'thriller', 'scifi', 'imax']

In [ ]:
#ต้องการเอา query ไปเทียบหา cosine_similarity กับทีละบรรทัดในดาต้าเฟรม
แปลว่าต้อง vectorize ทีละบรรทัดของ allgenres แล้วค่อยเทียบ
และ likedbinary.shape (7,19) ต้องทำให้เหลือ (1,19) ใช่ไหม ถึงจะเอามาเทียบกับแต่ละบรรทัดของดาต้าเฟรมได้

In [43]:
allgenres

0       Adventure|Animation|Children|Comedy|Fantasy
1                        Adventure|Children|Fantasy
2                                    Comedy|Romance
3                              Comedy|Drama|Romance
4                                            Comedy
                           ...                     
9737                Action|Animation|Comedy|Fantasy
9738                       Animation|Comedy|Fantasy
9739                                          Drama
9740                               Action|Animation
9741                                         Comedy
Name: genres, Length: 9742, dtype: object

In [129]:
tfidf_matrix

<9742x19 sparse matrix of type '<class 'numpy.float64'>'
	with 22050 stored elements in Compressed Sparse Row format>

In [86]:
likedbinary

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

In [38]:
likedbinary.shape

(7, 19)

In [87]:
from sklearn.metrics.pairwise import cosine_similarity

cs = cosine_similarity(tfidf_matrix,likedbinary)
print (cs)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.57860574 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [88]:
cs.shape

(9742, 7)

In [89]:
cs[0:6]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.54932817, 0.        , 0.        , 0.        , 0.54204235,
        0.        , 0.        ]])

In [36]:
from sklearn.metrics.pairwise import cosine_similarity

cs2 = cosine_similarity(tfidf_matrix[0],likedbinary)
cs2

array([[0., 0., 0., 0., 0., 0., 0.]])

In [37]:
cs2.shape

(1, 7)

In [ ]:
#อะไรวะ?

In [90]:
from sklearn.metrics.pairwise import cosine_similarity

csT = cosine_similarity(likedbinary,tfidf_matrix)
print (csT)

[[0.         0.         0.         ... 0.         0.57860574 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [91]:
csT.shape

(7, 9742)

In [92]:
csT[0:6]

array([[0.        , 0.        , 0.        , ..., 0.        , 0.57860574,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [93]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, likedbinary)
cosine_sim

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.57860574, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [94]:
cosine_sim.shape

(9742, 7)

In [95]:
cosine_sim[0:6]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.54932817, 0.        , 0.        , 0.        , 0.54204235,
        0.        , 0.        ]])

In [ ]:
testVectorizerArray = vectorizer.transform(test_set).toarray()

In [54]:
countlike = CountVectorizer()
countlikevec = countlike.fit_transform(flat_list)
countlikevec

<7x5 sparse matrix of type '<class 'numpy.int64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [27]:
countlikevec2 = countlike.transform(flat_list).toarray()
countlikevec2


array([[1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0]])

In [ ]:
likedbinary = lb.transform(lowercase) #this is test set? or just flat_list?

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])



In [ ]:
cosine_similarities = linear_kernel(tfidf[0:1], tfidf).flatten()

In [36]:
from sklearn.metrics.pairwise import linear_kernel

cosine_similarities = linear_kernel(likedbinary, tfidf_matrix).flatten()
cosine_similarities

array([0., 0., 0., ..., 0., 0., 0.])

In [37]:
cosine_similarities.shape

(68194,)

In [40]:
cosine_similarities2 = linear_kernel(likedbinary, tfidf_matrix)
cosine_similarities2

array([[0.        , 0.        , 0.        , ..., 0.        , 0.57860574,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [41]:
cosine_similarities2.shape

(7, 9742)

In [29]:
tfidf_matrix

<9742x19 sparse matrix of type '<class 'numpy.float64'>'
	with 22050 stored elements in Compressed Sparse Row format>

In [30]:
pd.DataFrame(tfidf_matrix.todense())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.000000,0.416846,0.516225,0.504845,0.267586,0.0,0.0,0.000000,0.482990,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.000000,0.512361,0.000000,0.620525,0.000000,0.0,0.0,0.000000,0.593662,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.570915,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.505015,0.0,0.0,0.466405,0.000000,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,0.436010,0.000000,0.614603,0.000000,0.318581,0.0,0.0,0.000000,0.575034,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9738,0.000000,0.000000,0.682937,0.000000,0.354002,0.0,0.0,0.000000,0.638968,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9739,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9740,0.578606,0.000000,0.815607,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [38]:
9742*19

185098

In [39]:
9742*7

68194

In [ ]:
ต้องการ 
(9742, 1)

In [43]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrixx = tfidf_vectorizer.fit_transform(allgenres)
print (tfidf_matrixx)
cosine = cosine_similarity(tfidf_matrixx[length-1], likedbinary)
print (cosine)

  (0, 8)	0.482990142708577
  (0, 4)	0.26758647689140014
  (0, 3)	0.5048454681396087
  (0, 2)	0.5162254711770092
  (0, 1)	0.41684567364693936
  (1, 8)	0.5936619434123594
  (1, 3)	0.620525172745643
  (1, 1)	0.5123612074824268
  (2, 14)	0.8210088907493954
  (2, 4)	0.5709154064399099
  (3, 7)	0.46640480307738325
  (3, 14)	0.726240982959826
  (3, 4)	0.5050154397005037
  (4, 4)	1.0
  (5, 16)	0.5420423542868653
  (5, 5)	0.6359470441562756
  (5, 0)	0.5493281743985542
  (6, 14)	0.8210088907493954
  (6, 4)	0.5709154064399099
  (7, 3)	0.7711121633813997
  (7, 1)	0.6366993258087036
  (8, 0)	1.0
  (9, 16)	0.5457299419583338
  (9, 0)	0.5530653284926609
  (9, 1)	0.6295217016667962
  :	:
  (9731, 15)	0.5912169125932074
  (9731, 0)	0.47946635727353987
  (9731, 4)	0.3503331832902893
  (9731, 1)	0.5457483258721835
  (9732, 15)	0.5491808953132112
  (9732, 0)	0.4453758980017919
  (9732, 4)	0.32542420076143624
  (9732, 2)	0.6278054979536716
  (9733, 7)	0.43179372819853856
  (9733, 2)	0.9019723811117537
  (9

NameError: name 'length' is not defined

In [184]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrixx = tfidf_vectorizer.fit_transform(allgenres)
print (tfidf_matrixx)
cosine = cosine_similarity(tfidf_matrixx[length-1], likedbinary)
print (cosine)

  (0, 8)	0.482990142708577
  (0, 4)	0.26758647689140014
  (0, 3)	0.5048454681396087
  (0, 2)	0.5162254711770092
  (0, 1)	0.41684567364693936
  (1, 8)	0.5936619434123594
  (1, 3)	0.620525172745643
  (1, 1)	0.5123612074824268
  (2, 14)	0.8210088907493954
  (2, 4)	0.5709154064399099
  (3, 7)	0.46640480307738325
  (3, 14)	0.726240982959826
  (3, 4)	0.5050154397005037
  (4, 4)	1.0
  (5, 16)	0.5420423542868653
  (5, 5)	0.6359470441562756
  (5, 0)	0.5493281743985542
  (6, 14)	0.8210088907493954
  (6, 4)	0.5709154064399099
  (7, 3)	0.7711121633813997
  (7, 1)	0.6366993258087036
  (8, 0)	1.0
  (9, 16)	0.5457299419583338
  (9, 0)	0.5530653284926609
  (9, 1)	0.6295217016667962
  :	:
  (9731, 15)	0.5912169125932074
  (9731, 0)	0.47946635727353987
  (9731, 4)	0.3503331832902893
  (9731, 1)	0.5457483258721835
  (9732, 15)	0.5491808953132112
  (9732, 0)	0.4453758980017919
  (9732, 4)	0.32542420076143624
  (9732, 2)	0.6278054979536716
  (9733, 7)	0.43179372819853856
  (9733, 2)	0.9019723811117537
  (9

NameError: name 'length' is not defined

In [248]:
cosine.shape

(9741, 7)

In [249]:
cosine[200:250]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.61569562, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0. 

In [44]:
tfidf_matrixx

<9742x19 sparse matrix of type '<class 'numpy.float64'>'
	with 22050 stored elements in Compressed Sparse Row format>

In [49]:
cosine = cosine_similarity(tfidf_matrixx[:-1], likedbinary)
cosine

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.57860574, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [187]:
cosine[0]

array([0., 0., 0., 0., 0., 0., 0.])

In [189]:
cosine[9740]

array([0.57860574, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ])

In [50]:
cosine.shape

(9741, 7)

In [55]:
pip install --user -U nltk

     |████████████████████████████████| 1.4 MB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 738 kB 14.1 MB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434672 sha256=57d31049ceba30196aaf6edf995b4db7cbc003b945cb79c5506fb9c224e3bcff
  Stored in directory: /home/jovyan/.cache/pip/wheels/ff/d5/7b/f1fb4e1e1603b2f01c2424dd60fbcc50c12ef918bafc44b155
Successfully built nltk
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [39]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import nltk
from nltk.corpus import stopwords
import numpy as np
import numpy.linalg as LA

train_set = ["The sky is blue.", "The sun is bright."] #Documents
test_set = ["The sun in the sky is bright."] #Query
stopWords = stopwords.words('english')

vectorizee = CountVectorizer(stop_words = stopWords)
#print vectorizer
transformer = TfidfTransformer()
#print transformer

trainVectorizerArray = vectorizee.fit_transform(train_set).toarray()
testVectorizerArray = vectorizee.transform(test_set).toarray()
print ('Fit Vectorizer to train set', trainVectorizerArray)
print ('Transform Vectorizer to test set', testVectorizerArray)
cx = lambda a, b : round(np.inner(a, b)/(LA.norm(a)*LA.norm(b)), 3)

for vector in trainVectorizerArray:
    print (vector)
    for testV in testVectorizerArray:
        print (testV)
        cosine = cx(vector, testV)
        print (cosine)

print (transformer.fit(trainVectorizerArray))
print (transformer.transform(trainVectorizerArray).toarray())

transformer.fit(testVectorizerArray)
print (tfidf = transformer.transform(testVectorizerArray))
print (tfidf.todense())

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/home/jovyan/nltk_data'
    - '/opt/conda/nltk_data'
    - '/opt/conda/share/nltk_data'
    - '/opt/conda/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [40]:
trainVectorizerArray = vectorizer.fit_transform(allgenres)
testVectorizerArray = vectorizer.transform(lowercase)
print ('Fit Vectorizer to train set', trainVectorizerArray)
print ('Transform Vectorizer to test set', testVectorizerArray)
cx = lambda a, b : round(np.inner(a, b)/(LA.norm(a)*LA.norm(b)), 3)


Fit Vectorizer to train set   (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 8)	1
  (1, 1)	1
  (1, 3)	1
  (1, 8)	1
  (2, 4)	1
  (2, 14)	1
  (3, 4)	1
  (3, 14)	1
  (3, 7)	1
  (4, 4)	1
  (5, 0)	1
  (5, 5)	1
  (5, 16)	1
  (6, 4)	1
  (6, 14)	1
  (7, 1)	1
  (7, 3)	1
  (8, 0)	1
  (9, 1)	1
  (9, 0)	1
  (9, 16)	1
  :	:
  (9731, 1)	1
  (9731, 4)	1
  (9731, 0)	1
  (9731, 15)	1
  (9732, 2)	1
  (9732, 4)	1
  (9732, 0)	1
  (9732, 15)	1
  (9733, 2)	1
  (9733, 7)	1
  (9734, 4)	1
  (9734, 7)	1
  (9735, 2)	1
  (9736, 6)	1
  (9737, 2)	1
  (9737, 4)	1
  (9737, 8)	1
  (9737, 0)	1
  (9738, 2)	1
  (9738, 4)	1
  (9738, 8)	1
  (9739, 7)	1
  (9740, 2)	1
  (9740, 0)	1
  (9741, 4)	1
Transform Vectorizer to test set   (0, 0)	1
  (1, 15)	1
  (2, 13)	1
  (3, 15)	1
  (4, 16)	1
  (5, 15)	1
  (6, 11)	1


In [42]:
vectorizee = CountVectorizer()


trainVectorizerArray = vectorizee.fit_transform(allgenres)
testVectorizerArray = vectorizee.transform(lowercase)
print ('Fit Vectorizer to train set', trainVectorizerArray)
print ('Transform Vectorizer to test set', testVectorizerArray)
cx = lambda a, b : round(np.inner(a, b)/(LA.norm(a)*LA.norm(b)), 3)

transformer = TfidfTransformer()

for vector in trainVectorizerArray:
    print (vector)
    for testV in testVectorizerArray:
        print (testV)
        cosine = cx(vector, testV)
        print (cosine)

print (transformer.fit(trainVectorizerArray))
print (transformer.transform(trainVectorizerArray).toarray())

transformer.fit(testVectorizerArray)
print (tfidf = transformer.transform(testVectorizerArray))
print (tfidf.todense())

Fit Vectorizer to train set   (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 8)	1
  (1, 1)	1
  (1, 3)	1
  (1, 8)	1
  (2, 4)	1
  (2, 14)	1
  (3, 4)	1
  (3, 14)	1
  (3, 7)	1
  (4, 4)	1
  (5, 0)	1
  (5, 5)	1
  (5, 16)	1
  (6, 4)	1
  (6, 14)	1
  (7, 1)	1
  (7, 3)	1
  (8, 0)	1
  (9, 1)	1
  (9, 0)	1
  (9, 16)	1
  :	:
  (9731, 1)	1
  (9731, 4)	1
  (9731, 0)	1
  (9731, 15)	1
  (9732, 2)	1
  (9732, 4)	1
  (9732, 0)	1
  (9732, 15)	1
  (9733, 2)	1
  (9733, 7)	1
  (9734, 4)	1
  (9734, 7)	1
  (9735, 2)	1
  (9736, 6)	1
  (9737, 2)	1
  (9737, 4)	1
  (9737, 8)	1
  (9737, 0)	1
  (9738, 2)	1
  (9738, 4)	1
  (9738, 8)	1
  (9739, 7)	1
  (9740, 2)	1
  (9740, 0)	1
  (9741, 4)	1
Transform Vectorizer to test set   (0, 0)	1
  (1, 15)	1
  (2, 13)	1
  (3, 15)	1
  (4, 16)	1
  (5, 15)	1
  (6, 11)	1
  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 8)	1
  (0, 0)	1


ValueError: dimension mismatch

In [144]:
allgenres

0       Adventure|Animation|Children|Comedy|Fantasy
1                        Adventure|Children|Fantasy
2                                    Comedy|Romance
3                              Comedy|Drama|Romance
4                                            Comedy
                           ...                     
9737                Action|Animation|Comedy|Fantasy
9738                       Animation|Comedy|Fantasy
9739                                          Drama
9740                               Action|Animation
9741                                         Comedy
Name: genres, Length: 9742, dtype: object

In [145]:
splittedgenres = [sub.split("|") for sub in allgenres]
splittedgenres

[['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adventure', 'Children', 'Fantasy'],
 ['Comedy', 'Romance'],
 ['Comedy', 'Drama', 'Romance'],
 ['Comedy'],
 ['Action', 'Crime', 'Thriller'],
 ['Comedy', 'Romance'],
 ['Adventure', 'Children'],
 ['Action'],
 ['Action', 'Adventure', 'Thriller'],
 ['Comedy', 'Drama', 'Romance'],
 ['Comedy', 'Horror'],
 ['Adventure', 'Animation', 'Children'],
 ['Drama'],
 ['Action', 'Adventure', 'Romance'],
 ['Crime', 'Drama'],
 ['Drama', 'Romance'],
 ['Comedy'],
 ['Comedy'],
 ['Action', 'Comedy', 'Crime', 'Drama', 'Thriller'],
 ['Comedy', 'Crime', 'Thriller'],
 ['Crime', 'Drama', 'Horror', 'Mystery', 'Thriller'],
 ['Action', 'Crime', 'Thriller'],
 ['Drama', 'SciFi'],
 ['Drama', 'Romance'],
 ['Drama'],
 ['Children', 'Drama'],
 ['Drama', 'Romance'],
 ['Adventure', 'Drama', 'Fantasy', 'Mystery', 'SciFi'],
 ['Crime', 'Drama'],
 ['Drama'],
 ['Mystery', 'SciFi', 'Thriller'],
 ['Children', 'Drama'],
 ['Crime', 'Drama'],
 ['Children', 'Comedy'],
 ['C

In [49]:
lowerallgenres = [item.lower() for item in splittedgenres.flatten()]
lowerallgenres

AttributeError: 'list' object has no attribute 'flatten'

In [50]:
splittedgenres

[['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adventure', 'Children', 'Fantasy'],
 ['Comedy', 'Romance'],
 ['Comedy', 'Drama', 'Romance'],
 ['Comedy'],
 ['Action', 'Crime', 'Thriller'],
 ['Comedy', 'Romance'],
 ['Adventure', 'Children'],
 ['Action'],
 ['Action', 'Adventure', 'Thriller'],
 ['Comedy', 'Drama', 'Romance'],
 ['Comedy', 'Horror'],
 ['Adventure', 'Animation', 'Children'],
 ['Drama'],
 ['Action', 'Adventure', 'Romance'],
 ['Crime', 'Drama'],
 ['Drama', 'Romance'],
 ['Comedy'],
 ['Comedy'],
 ['Action', 'Comedy', 'Crime', 'Drama', 'Thriller'],
 ['Comedy', 'Crime', 'Thriller'],
 ['Crime', 'Drama', 'Horror', 'Mystery', 'Thriller'],
 ['Action', 'Crime', 'Thriller'],
 ['Drama', 'SciFi'],
 ['Drama', 'Romance'],
 ['Drama'],
 ['Children', 'Drama'],
 ['Drama', 'Romance'],
 ['Adventure', 'Drama', 'Fantasy', 'Mystery', 'SciFi'],
 ['Crime', 'Drama'],
 ['Drama'],
 ['Mystery', 'SciFi', 'Thriller'],
 ['Children', 'Drama'],
 ['Crime', 'Drama'],
 ['Children', 'Comedy'],
 ['C

In [51]:
len(splittedgenres)

9742

In [62]:
hmm = CountVectorizer(token_pattern='(?u)\\b\\w+\\b')
vecbag = []
for item in splittedgenres:
    hmmvec= hmm.fit_transform(item)
    vecbag.append(hmmvec)
vecbag

[<5x5 sparse matrix of type '<class 'numpy.int64'>'
 	with 5 stored elements in Compressed Sparse Row format>,
 <3x3 sparse matrix of type '<class 'numpy.int64'>'
 	with 3 stored elements in Compressed Sparse Row format>,
 <2x2 sparse matrix of type '<class 'numpy.int64'>'
 	with 2 stored elements in Compressed Sparse Row format>,
 <3x3 sparse matrix of type '<class 'numpy.int64'>'
 	with 3 stored elements in Compressed Sparse Row format>,
 <1x1 sparse matrix of type '<class 'numpy.int64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 <3x3 sparse matrix of type '<class 'numpy.int64'>'
 	with 3 stored elements in Compressed Sparse Row format>,
 <2x2 sparse matrix of type '<class 'numpy.int64'>'
 	with 2 stored elements in Compressed Sparse Row format>,
 <2x2 sparse matrix of type '<class 'numpy.int64'>'
 	with 2 stored elements in Compressed Sparse Row format>,
 <1x1 sparse matrix of type '<class 'numpy.int64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 

In [63]:
pd.DataFrame(vecbag.todense())

AttributeError: 'list' object has no attribute 'todense'

In [64]:
print(vecbag[0]) #how is this 5*5 matrix?

  (0, 0)	1
  (1, 1)	1
  (2, 2)	1
  (3, 3)	1
  (4, 4)	1


In [65]:
print(vecbag[1])

  (0, 0)	1
  (1, 1)	1
  (2, 2)	1


In [68]:
tf = TfidfVectorizer(analyzer='word',stop_words='english')
vecbag2 = []
for item in splittedgenres:
    tfvec= tf.fit_transform(item)
    vecbag2.append(tfvec)
vecbag2

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [120]:
vectorizer.get_feature_names()

['action',
 'adventure',
 'animation',
 'children',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'fantasy',
 'filmnoir',
 'horror',
 'imax',
 'musical',
 'mystery',
 'romance',
 'scifi',
 'thriller',
 'war',
 'western']

In [122]:
vocab = tfidf_vectorizer.get_feature_names()
vocab

['action',
 'adventure',
 'animation',
 'children',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'fantasy',
 'filmnoir',
 'horror',
 'imax',
 'musical',
 'mystery',
 'romance',
 'scifi',
 'thriller',
 'war',
 'western']

In [ ]:
tfidf_matrix = tf.fit_transform(all_movies['genres']) #<--argument is Series


cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
def genre_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    
ตัวอย่างนี้เทียบ tfidf_matrix กับตัวมันเอง ก็คือบรรทัดที่ 1 เทียบกับทุกบรรทัด บรรทัดที่ 2 เทียบกับทุกบรรทัด X เป็น argument ตัวเดียว
แต่ของเราเนี่ย ตอนนี้เป็นลิสท์ คือ lowercase เทียบกับ sublist ในลิสท์ใหญ่อีกทีนึง ทำไมมันไม่เหมือนกันล่ะ?

In [ ]:
squeeze 7 rows of binaries values into one row

In [ ]:
7*5 labelbinarized into 7*19 and now have to squeeze somehow the 7 rows to 1

In [ ]:
then it will be of shape 1*19 and then i can compare it with each row

In [ ]:
which has to be in shape of (1,19) which should not be difficult because it is one movie per row
and i just need to labelbinarize each of them

In [ ]:
รู้สึกเข้าใจวิธีคิดในการเขียนโปรแกรมขึ้นมาเลย ถ้าจะทำให้เห็นเป็นชิ้นเป็นอัน แบบของน้องคนนั้นที่ทำเปลี่ยนสีฟ้อนท์แว้บๆ ของเราคือให้ recommend ออกมา
คือให้เซ็ทอัพ distributed system ในการดึงรูปเข้ามาป้อนโปรแกรม เพื่อรีเทรน
แต่จะให้ใครทำ เออ อาจจะให้น้องใหม่ทำ เพราะมันเป็นระบบที่ต้องเซอร์วิสให้หุ่นของบริษัท
ส่วนฟรีแลนซ์ก็ให้ทำอะไรที่มันเสี่ยงๆหน่อย ไอเดียใหม่ หรือจะไปทางไฟแนนซ์ แล้ว Obodroid เกี่ยวไหมไฟแนนซ์

In [69]:
likedbinary

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

In [70]:
likedbinary.squeeze()

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

In [72]:
#tfidf_matrix = tf.fit_transform(all_movies['genres']) #<--argument is Series
tfidf_matrixa = tf.fit_transform(flat_list)
tfidf_matrixa

<7x5 sparse matrix of type '<class 'numpy.float64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [90]:
matrixa = pd.DataFrame(tfidf_matrixa.todense())
matrixa

,0,1,2,3,4
0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0
5,0.0,0.0,0.0,1.0,0.0
6,0.0,1.0,0.0,0.0,0.0


In [75]:
tfidf_matrix

<9742x19 sparse matrix of type '<class 'numpy.float64'>'
	with 22050 stored elements in Compressed Sparse Row format>

In [78]:
matrixb = pd.DataFrame(tfidf_matrix.todense())
matrixb

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.000000,0.416846,0.516225,0.504845,0.267586,0.0,0.0,0.000000,0.482990,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.000000,0.512361,0.000000,0.620525,0.000000,0.0,0.0,0.000000,0.593662,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.570915,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.505015,0.0,0.0,0.466405,0.000000,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,0.436010,0.000000,0.614603,0.000000,0.318581,0.0,0.0,0.000000,0.575034,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9738,0.000000,0.000000,0.682937,0.000000,0.354002,0.0,0.0,0.000000,0.638968,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9739,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9740,0.578606,0.000000,0.815607,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [81]:
from sklearn.metrics.pairwise import linear_kernel

linear_kernel(likedbinary, matrixb)

array([[0.        , 0.        , 0.        , ..., 0.        , 0.57860574,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [82]:
linear_kernel(likedbinary, matrixb).shape

(7, 9742)

In [76]:
lowercase

['action', 'scifi', 'mystery', 'scifi', 'thriller', 'scifi', 'imax']

In [77]:
likedbinary

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

In [88]:
pd.DataFrame(vectors.todense())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9738,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9739,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
9740,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [87]:
pd.DataFrame(likedbinary) #numpy.array cannot todense()
#เราทำให้ dataframe เหลือคอลัมน์เดียวได้โดยการ เอา todense() ออก
#แล้วเราทำไห้ dataframe เหลือบรรทัดเดียวได้ไหม?

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [91]:
print(vectors.toarray())

[[0 1 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [92]:
vectors.toarray().shape

(9742, 19)

In [94]:
likedbinary.shape

(7, 19)

In [133]:
splittedgenres[0]

['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']

In [99]:
bag =[]
for item in splittedgenres[0]:
    item.lower()
    bag.append(item)

In [100]:
bag

['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']

In [96]:
lowercase

['action', 'scifi', 'mystery', 'scifi', 'thriller', 'scifi', 'imax']

In [111]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(handle_unknown='ignore')
dearyme = onehotencoder.fit(likedbinary)
dearyme

OneHotEncoder(handle_unknown='ignore')

In [115]:
dearyma = onehotencoder.fit(lowercase)
dearyma

ValueError: Expected 2D array, got 1D array instead:
array=['action' 'scifi' 'mystery' 'scifi' 'thriller' 'scifi' 'imax'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [102]:
from sklearn.metrics.pairwise import cosine_similarity

cs3 = cosine_similarity(splittedgenres[0], lowercase)
cs3

ValueError: could not convert string to float: 'Adventure'

In [117]:
pip install gensim

     |████████████████████████████████| 24.2 MB 122 kB/s  eta 0:00:01   |███▉                            | 2.9 MB 4.4 MB/s eta 0:00:05     |████████████                    | 9.0 MB 4.4 MB/s eta 0:00:04     |█████████████▋                  | 10.3 MB 4.4 MB/s eta 0:00:04     |██████████████▌                 | 10.9 MB 4.4 MB/s eta 0:00:04     |█████████████████               | 12.8 MB 14.7 MB/s eta 0:00:01     |██████████████████████▋         | 17.1 MB 14.7 MB/s eta 0:00:01     |█████████████████████████▌      | 19.3 MB 14.7 MB/s eta 0:00:01     |██████████████████████████▌     | 20.0 MB 14.7 MB/s eta 0:00:01     |█████████████████████████████▏  | 22.1 MB 14.7 MB/s eta 0:00:01
     |████████████████████████████████| 119 kB 16.1 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-4.2.0-py3-none-any.whl size=109632 sha256=17e73c283960323de6a3f9a494c38660a3b80193d29fd53c53b495680ad83967
  Stored in directory: /home/jovyan/.cache/pip/wheels/24/f6/ea/70a0761bdfaeacff66662751fe

In [118]:
from gensim.models import Word2Vec

In [119]:
 model = Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)

NameError: name 'sentences' is not defined

In [123]:
model = Word2Vec(vocab, workers=4)
model

In [124]:
my_dict = dict({})
for idx, key in enumerate(model.wv.vocab):
    my_dict[key] = model.wv[key]
    # Or my_dict[key] = model.wv.get_vector(key)
    # Or my_dict[key] = model.wv.word_vec(key, use_norm=False)
my_dict

{'a': array([ 1.0815460e-03,  1.1569420e-03,  2.4339068e-03, -1.6087089e-03,
        -2.5664375e-03, -4.7423020e-03, -1.0537311e-03,  3.6003923e-03,
         8.2367966e-05, -1.1128886e-03,  1.6436288e-03, -2.8645799e-03,
         4.4876337e-03, -4.5265113e-03, -2.8602909e-03,  3.0680040e-03,
         4.0316931e-03,  2.2267040e-03,  2.4376432e-03,  3.1403973e-04,
         1.3202528e-03,  4.7355485e-03, -9.7929954e-04,  3.6607760e-03,
         2.4974020e-03,  2.5863932e-03, -4.9687605e-03, -1.5316664e-03,
        -2.2718944e-03,  3.5080567e-03, -3.8550186e-03,  3.7882992e-03,
        -3.3436343e-03, -4.0401295e-03,  2.3295044e-03, -4.8826253e-03,
         4.0439633e-03,  3.7876267e-03,  4.0363041e-03, -2.5030877e-03,
         4.2215381e-03,  1.1200885e-03,  3.1542600e-04, -7.0953800e-04,
         3.4586031e-03, -4.3243533e-03,  4.0252181e-03,  2.7323733e-04,
        -4.3244730e-03,  3.9216150e-03,  2.4744400e-03, -4.5647812e-03,
         2.1088242e-03,  9.9405285e-04, -5.2092125e-04,  4.

In [125]:
trial = Word2Vec(lowercase)
trial

In [127]:
my_dict2 = dict({})
for idx, key in enumerate(trial.wv.vocab):
    my_dict2[key] = trial.wv[key]
    # Or my_dict[key] = model.wv.get_vector(key)
my_dict2

{'i': array([ 1.7606876e-03,  2.0191965e-03, -4.4726194e-03,  9.9115388e-04,
         2.0650676e-03,  4.7796071e-03,  4.4158702e-03,  4.0349918e-03,
         8.3751528e-04,  1.5656111e-03, -2.8585822e-03,  4.1617504e-03,
         2.6964708e-03, -4.3118703e-03,  4.2532277e-03, -3.5224790e-03,
        -4.4275662e-03, -3.7802218e-03,  3.9435169e-03, -6.7533302e-04,
         4.4429745e-03, -1.8638581e-03, -2.6812912e-03, -2.2771279e-03,
        -2.4098214e-03, -2.7613196e-04, -1.1383059e-03, -4.7354489e-03,
         4.0422599e-03, -1.8947364e-03, -3.7667439e-03, -3.8337123e-03,
        -3.6545859e-03,  2.0310828e-04,  1.3434535e-03, -3.3891862e-03,
        -2.9127591e-03, -1.5262832e-03, -4.1930242e-03, -1.6121399e-03,
        -9.8923547e-04,  1.1957657e-03, -3.3787899e-03, -1.3697805e-03,
        -4.3906718e-03, -4.6019717e-03,  5.0679169e-04,  1.9625854e-03,
        -4.2553677e-04, -4.0360871e-03,  4.8933122e-03,  3.6843487e-03,
         3.7253262e-03,  4.2109513e-03,  2.0683061e-04, -2.

In [130]:
from gensim.models import Word2Vec

sentences = [["cat", "say", "meow"], ["dog", "say", "woof"]]

model = Word2Vec(min_count=1)

model.build_vocab(sentences)  # prepare the model vocabulary

model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)  # train word vectors

(1, 30)

In [131]:
print(model.train)

<bound method Word2Vec.train of <gensim.models.word2vec.Word2Vec object at 0x7ff41591f9a0>>


In [132]:
model.train.shape

AttributeError: 'function' object has no attribute 'shape'

In [135]:
from gensim.models import Word2Vec

# Take some sample sentences
tokenized_sentences = [["here","is","one"],["and","here","is","another"]]

# Initialise model, for more information, please check the Gensim Word2vec documentation
model = Word2Vec(tokenized_sentences, size=100, window=2, min_count=0)

# Get a list of words in the vocabulary
words = model.wv.vocab.keys()

# Make a dictionary
we_dict = {word:model.wv[word] for word in words}
we_dict

{'here': array([-1.9750337e-03, -2.9658661e-03,  2.2372971e-03, -1.8308554e-03,
         1.6861717e-03,  7.0699543e-04,  3.7243771e-03,  1.3265229e-03,
        -4.7945813e-03,  2.6180693e-03, -3.9571319e-03,  2.2168672e-03,
        -4.3444396e-03,  3.3723228e-04, -3.2985241e-03,  1.2283511e-03,
         3.4749941e-03,  1.0351891e-03, -2.1958840e-03, -2.3156442e-03,
        -1.3630793e-03, -8.7992882e-04, -1.9395554e-03, -1.5673682e-03,
        -3.7146311e-03, -4.9161320e-03,  4.1895341e-03,  2.7029370e-03,
         2.8072943e-03,  2.7480363e-04,  1.0244113e-03,  4.0994645e-03,
        -1.8623518e-03,  7.9925411e-04, -1.3401848e-03, -3.4499848e-03,
        -4.6155835e-03, -4.5900489e-04, -8.9476648e-04,  4.0650810e-03,
        -6.8655686e-04, -3.8968273e-03, -2.9246511e-03,  1.9472447e-03,
         3.3079060e-03, -4.6033696e-03, -2.6797380e-03, -3.0843301e-03,
         7.1088631e-05, -4.9921754e-03, -3.3960601e-03,  1.4576574e-03,
        -1.5530566e-04,  2.7717482e-03,  2.3619435e-03, 

In [ ]:
customers = df["CustomerID"].unique().tolist()

In [149]:
movies = all_movies['title'].unique().tolist()
movies

['Toy Story (1995)',
 'Jumanji (1995)',
 'Grumpier Old Men (1995)',
 'Waiting to Exhale (1995)',
 'Father of the Bride Part II (1995)',
 'Heat (1995)',
 'Sabrina (1995)',
 'Tom and Huck (1995)',
 'Sudden Death (1995)',
 'GoldenEye (1995)',
 'American President, The (1995)',
 'Dracula: Dead and Loving It (1995)',
 'Balto (1995)',
 'Nixon (1995)',
 'Cutthroat Island (1995)',
 'Casino (1995)',
 'Sense and Sensibility (1995)',
 'Four Rooms (1995)',
 'Ace Ventura: When Nature Calls (1995)',
 'Money Train (1995)',
 'Get Shorty (1995)',
 'Copycat (1995)',
 'Assassins (1995)',
 'Powder (1995)',
 'Leaving Las Vegas (1995)',
 'Othello (1995)',
 'Now and Then (1995)',
 'Persuasion (1995)',
 'City of Lost Children, The (Cité des enfants perdus, La) (1995)',
 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 'Dangerous Minds (1995)',
 'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)',
 'Babe (1995)',
 'Dead Man Walking (1995)',
 'It Takes Two (1995)',
 'Clueless (1995)',
 'Cry, the Beloved Country

In [ ]:
random.shuffle(customers)

# extract 90% of customer ID's
customers_train = [customers[i] for i in range(round(0.9*len(customers)))]

# split data into train and validation set
train_df = df[df['CustomerID'].isin(customers_train)]
validation_df = df[~df['CustomerID'].isin(customers_train)]

In [151]:
import random
random.shuffle(movies)
movie_train = [movies[i] for i in range(round(0.9*len(movies)))]
movie_train


["Fathers' Day (1997)",
 'Inside Llewyn Davis (2013)',
 'Wuthering Heights (1939)',
 'Mummy: Tomb of the Dragon Emperor, The (2008)',
 'The Girls (1961)',
 "There's Something About Mary (1998)",
 "My Boss's Daughter (2003)",
 'Atomic Blonde (2017)',
 'Henry V (1989)',
 "Monty Python's The Meaning of Life (1983)",
 'Judge Dredd (1995)',
 'Benchwarmers, The (2006)',
 'Behind Enemy Lines II: Axis of Evil (2006)',
 'Haunting in Connecticut, The (2009)',
 "Charlotte's Web (2006)",
 "Big Momma's House (2000)",
 'Sympathy for the Devil (1968)',
 'Vertigo (1958)',
 'Confessions of a Shopaholic (2009)',
 'Tucker: The Man and His Dream (1988)',
 'Gleaners & I, The (Les glaneurs et la glaneuse) (2000)',
 'Clockers (1995)',
 'Go Figure (2005)',
 '20 Feet from Stardom (Twenty Feet from Stardom) (2013)',
 'Harry Potter and the Deathly Hallows: Part 2 (2011)',
 'Phil Spector (2013)',
 "Teacher's Pet (1958)",
 'You Will Meet a Tall Dark Stranger (2010)',
 'Man from Snowy River, The (1982)',
 'Blast fr

In [138]:
splittedgenres

[['Crime', 'Horror', 'Mystery', 'Thriller'],
 ['Drama'],
 ['Crime', 'Drama'],
 ['Action', 'Thriller'],
 ['Comedy', 'Fantasy'],
 ['Documentary'],
 ['Comedy'],
 ['Adventure', 'Comedy', 'Drama', 'Fantasy'],
 ['Adventure', 'Animation', 'Children', 'Comedy', 'Musical'],
 ['Drama', 'Romance'],
 ['Comedy', 'Romance'],
 ['Documentary'],
 ['Children', 'Comedy'],
 ['Comedy'],
 ['Crime', 'Drama', 'Horror', 'Mystery'],
 ['Comedy', 'Drama', 'Romance'],
 ['Musical'],
 ['Drama'],
 ['Comedy', 'Crime'],
 ['Drama'],
 ['Documentary'],
 ['Horror'],
 ['Comedy'],
 ['Action', 'Adventure', 'Children'],
 ['Adventure', 'Romance', 'SciFi', 'IMAX'],
 ['Adventure', 'Animation', 'Children'],
 ['Comedy'],
 ['Comedy'],
 ['Drama'],
 ['Documentary'],
 ['Drama'],
 ['Crime', 'Mystery', 'Thriller'],
 ['Horror', 'SciFi'],
 ['Action', 'Drama', 'War'],
 ['Adventure', 'SciFi'],
 ['Comedy'],
 ['Comedy', 'Musical', 'Romance'],
 ['Comedy', 'Fantasy', 'Romance'],
 ['Animation', 'Children', 'Comedy'],
 ['Mystery', 'Thriller'],
 ['

In [140]:
all_movies

,index,movieId,title,genres
0,0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,2,Jumanji (1995),Adventure|Children|Fantasy
2,2,3,Grumpier Old Men (1995),Comedy|Romance
3,3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...,...
9737,9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,9739,193585,Flint (2017),Drama
9740,9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [146]:
all_movies['splittedgenres'] = splittedgenres
all_movies

,index,movieId,title,genres,splittedgenres
0,0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,1,2,Jumanji (1995),Adventure|Children|Fantasy,"[Adventure, Children, Fantasy]"
2,2,3,Grumpier Old Men (1995),Comedy|Romance,"[Comedy, Romance]"
3,3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"[Comedy, Drama, Romance]"
4,4,5,Father of the Bride Part II (1995),Comedy,[Comedy]
...,...,...,...,...,...
9737,9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,"[Action, Animation, Comedy, Fantasy]"
9738,9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,"[Animation, Comedy, Fantasy]"
9739,9739,193585,Flint (2017),Drama,[Drama]
9740,9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,"[Action, Animation]"


In [153]:
#train_df = df[df['CustomerID'].isin(customers_train)]
train_df = all_movies[all_movies['title'].isin(movie_train)]
train_df

,index,movieId,title,genres,splittedgenres
0,0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,1,2,Jumanji (1995),Adventure|Children|Fantasy,"[Adventure, Children, Fantasy]"
2,2,3,Grumpier Old Men (1995),Comedy|Romance,"[Comedy, Romance]"
3,3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"[Comedy, Drama, Romance]"
4,4,5,Father of the Bride Part II (1995),Comedy,[Comedy]
...,...,...,...,...,...
9737,9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,"[Action, Animation, Comedy, Fantasy]"
9738,9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,"[Animation, Comedy, Fantasy]"
9739,9739,193585,Flint (2017),Drama,[Drama]
9740,9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,"[Action, Animation]"


In [ ]:
splittedgenres = [sub.split("|") for sub in allgenres]


In [156]:
genres_train = []
for i in movie_train:
    temp = train_df[train_df['title'] == i]['genres'].tolist()
    genres_train.append(temp)
print(genres_train)

[['Comedy'], ['Drama'], ['Drama|Romance'], ['Action|Adventure|Fantasy|Thriller'], ['Comedy|Romance'], ['Comedy|Romance'], ['Comedy|Romance'], ['Thriller'], ['Action|Drama|Romance|War'], ['Comedy'], ['Action|Crime|SciFi'], ['Comedy'], ['Action|Thriller|War'], ['Horror|Thriller'], ['Children|Comedy|Drama|Fantasy'], ['Comedy'], ['Documentary|Musical'], ['Drama|Mystery|Romance|Thriller'], ['Comedy|Romance'], ['Drama'], ['Documentary'], ['Crime|Drama|Mystery'], ['Children|Comedy|Drama'], ['Documentary'], ['Action|Adventure|Drama|Fantasy|Mystery|IMAX'], ['Drama'], ['Comedy|Romance'], ['Comedy|Romance'], ['Drama|Romance|Western'], ['Comedy|Romance'], ['SciFi|Thriller|IMAX'], ['Comedy|Drama|Romance'], ['Drama'], ['Drama|Thriller'], ['Comedy|Drama'], ['Comedy|Drama|Romance'], ['Crime|Drama|Thriller'], ['Comedy|Crime|Drama'], ['Horror'], ['Drama|Mystery'], ['Action|Animation|Crime'], ['Drama'], ['Documentary|War'], ['Drama|Thriller'], ['Crime|Drama|Thriller'], ['Thriller'], ['Adventure|Comedy'],

In [163]:
len(genres_train)

8763

In [164]:
np.unique(genres_train)

array([list(['(N/A)']), list(['Action']), list(['Action|Adventure']),
       list(['Action|Adventure|Animation']),
       list(['Action|Adventure|Animation|Children']),
       list(['Action|Adventure|Animation|Children|Comedy']),
       list(['Action|Adventure|Animation|Children|Comedy|Fantasy']),
       list(['Action|Adventure|Animation|Children|Comedy|IMAX']),
       list(['Action|Adventure|Animation|Children|Comedy|Romance']),
       list(['Action|Adventure|Animation|Children|Comedy|SciFi']),
       list(['Action|Adventure|Animation|Children|Comedy|SciFi|IMAX']),
       list(['Action|Adventure|Animation|Children|Comedy|Western']),
       list(['Action|Adventure|Animation|Children|Fantasy']),
       list(['Action|Adventure|Animation|Children|Fantasy|SciFi']),
       list(['Action|Adventure|Animation|Comedy']),
       list(['Action|Adventure|Animation|Comedy|Fantasy|Mystery|SciFi']),
       list(['Action|Adventure|Animation|Comedy|Fantasy|SciFi']),
       list(['Action|Adventure|Anima

In [165]:
len(np.unique(genres_train))

912

In [166]:
8763*912

7991856

In [193]:
genres_train2 = flat_list

In [190]:
lowercase

['action', 'scifi', 'mystery', 'scifi', 'thriller', 'scifi', 'imax']

In [ ]:
purchases_train = []

# populate the list with the product codes
for i in tqdm(customers_train):
    temp = train_df[train_df["CustomerID"] == i]["StockCode"].tolist()
    purchases_train.append(temp)

# list to capture purchase history of the customers
purchases_val = []

# populate the list with the product codes
for i in tqdm(validation_df['CustomerID'].unique()):
    temp = validation_df[validation_df["CustomerID"] == i]["StockCode"].tolist()
    purchases_val.append(temp)

In [ ]:
# train word2vec model
model = Word2Vec(window = 10, sg = 1, hs = 0,
                 negative = 10, # for negative sampling
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14)

model.build_vocab(purchases_train, progress_per=200)

model.train(purchases_train, total_examples = model.corpus_count, 
            epochs=10, report_delay=1)

In [215]:
vocab = lowercase

In [216]:
vocab

['action', 'scifi', 'mystery', 'scifi', 'thriller', 'scifi', 'imax']

In [224]:
genres_train2

['Action', 'SciFi', 'Mystery', 'SciFi', 'Thriller', 'SciFi', 'IMAX']

In [197]:
model2 = Word2Vec(window = 10, sg = 1, hs = 0,
                 negative = 10, # for negative sampling
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14)

model2.build_vocab(genres_train2, progress_per=200)

model2.train(genres_train2, total_examples = model.corpus_count, 
            epochs=10, report_delay=1)

(0, 400)

In [198]:
model2.init_sims(replace=True)
print(model2)

Word2Vec(vocab=1, size=100, alpha=0.03)


In [217]:
X2 = model2[model2.wv.vocab]

X2.shape

<ipython-input-217-e8e860a5d5f9>:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  X2 = model2[model2.wv.vocab]


(1, 100)

In [223]:
X2

array([[ 0.01984781,  0.03615342,  0.05508519, -0.09833098,  0.03469901,
        -0.02027358,  0.06958389, -0.06052149, -0.14034535,  0.13804755,
         0.17570236,  0.06308728,  0.04472478, -0.02654674,  0.15193497,
        -0.1296846 ,  0.12551872,  0.02624963, -0.15778118, -0.08810411,
         0.08301982,  0.14840427,  0.02075593,  0.03325179, -0.01436393,
        -0.02768432,  0.02665904,  0.1032008 ,  0.1263216 , -0.07919533,
        -0.03568958, -0.14623594,  0.00523576, -0.11510266, -0.07988063,
         0.04693303, -0.12841545,  0.12865677,  0.04267652, -0.0319443 ,
         0.03996589, -0.14342532,  0.0596881 , -0.15124479,  0.14955497,
        -0.12307652,  0.17771351,  0.0753205 , -0.01093714, -0.0912469 ,
        -0.03186354,  0.14779079,  0.01446838, -0.0254302 ,  0.08131943,
         0.09841558,  0.02061726,  0.11048878,  0.01839558,  0.17091891,
        -0.07806698,  0.07534209,  0.02822581,  0.01418233, -0.14154136,
         0.07719857, -0.11487542, -0.13880108, -0.1

In [195]:
model = Word2Vec(window = 10, sg = 1, hs = 0,
                 negative = 10, # for negative sampling
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14)

model.build_vocab(genres_train, progress_per=200)

model.train(genres_train, total_examples = model.corpus_count, 
            epochs=10, report_delay=1)

(35092, 87680)

In [196]:
model.init_sims(replace=True)
print(model)

Word2Vec(vocab=222, size=100, alpha=0.03)


In [207]:
model2.init_sims(replace=True)
print(model2)

Word2Vec(vocab=1, size=100, alpha=0.03)


In [218]:
vocab

['action', 'scifi', 'mystery', 'scifi', 'thriller', 'scifi', 'imax']

In [159]:
# extract all vectors
X = model[model.wv.vocab]

X.shape

<ipython-input-159-882e57228d22>:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  X = model[model.wv.vocab]


(222, 100)

In [160]:
products = train_df[["title", "genres"]]

# remove duplicates
#products.drop_duplicates(inplace=True, subset='StockCode', keep="last")

# create product-ID and product-description dictionary
products_dict = products.groupby('genres')['title'].apply(list).to_dict()
products_dict

{'(N/A)': ['La cravate (1957)',
  'Pirates of the Caribbean: Dead Men Tell No Tales (2017)',
  'Superfast! (2015)',
  'Let It Be Me (1995)',
  'Trevor Noah: African American (2013)',
  'Guardians (2016)',
  'Green Room (2015)',
  'The Brand New Testament (2015)',
  'Hyena Road',
  'The Adventures of Sherlock Holmes and Doctor Watson',
  'A Cosmic Christmas (1977)',
  'Grease Live (2016)',
  'Noin 7 veljestä (1968)',
  'Paterson',
  'Ali Wong: Baby Cobra (2016)',
  "A Midsummer Night's Dream (2016)",
  'The Forbidden Dance (1990)',
  'Ethel & Ernest (2016)',
  'Whiplash (2013)',
  'The OA',
  'Lemonade (2016)',
  'Maria Bamford: Old Baby',
  'Death Note: Desu nôto (2006–2007)',
  'Generation Iron 2',
  'T2 3-D: Battle Across Time (1996)',
  'The Adventures of Sherlock Holmes and Doctor Watson: The Hunt for the Tiger (1980)',
  'The Putin Interviews (2017)',
  'Black Mirror',
  'Too Funny to Fail: The Life and Death of The Dana Carvey Show (2017)',
  'Serving in Silence: The Margarethe C

In [162]:
products_dict['Action|Adventure|Animation|Children|Comedy|SciFi']

['Chicken Little (2005)',
 'Meet the Robinsons (2007)',
 'Ratchet & Clank (2016)']

In [161]:
products_dict['The Lego Movie (2014)']

KeyError: 'The Lego Movie (2014)'

In [ ]:
What if we want to recommend products based on the multiple purchases he or she has made in the past?
https://www.analyticsvidhya.com/blog/2019/07/how-to-build-recommendation-system-word2vec-python/

In [219]:
def aggregate_vectors(products):
    product_vec = []
    for i in products:
        try:
            product_vec.append(model2[i])
        except KeyError:
            continue
        
    return np.mean(product_vec, axis=0)

In [220]:
aggregate_vectors(products)

<ipython-input-219-1567faade5ca>:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  product_vec.append(model2[i])


nan

In [204]:
products

,title,genres
0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,Jumanji (1995),Adventure|Children|Fantasy
2,Grumpier Old Men (1995),Comedy|Romance
3,Waiting to Exhale (1995),Comedy|Drama|Romance
4,Father of the Bride Part II (1995),Comedy
...,...,...
9737,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,Flint (2017),Drama
9740,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [221]:
model2[i]

<ipython-input-221-b3f2657c2131>:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  model2[i]


KeyError: "word 'Mystère à la Tour Eiffel (2015)' not in vocabulary"

In [247]:
vocab

['action', 'scifi', 'mystery', 'scifi', 'thriller', 'scifi', 'imax']

In [225]:
#vocab=['email','user','this','email','address','customer']
vectoriser3 = CountVectorizer().fit(vocab)
vectoriser3.vocabulary_ # show the word-matrix position pairs

{'action': 0, 'scifi': 3, 'mystery': 2, 'thriller': 4, 'imax': 1}

In [ ]:
# Analyse  list_1
list_1 = ['email','mail','address','netmail']
list_1_vect, list_1_cos = vect_cos(vectoriser, [' '.join(list_1)])

# Analyse list_2
list_2 = ['address','ip','network']
list_2_vect, list_2_cos = vect_cos(vectoriser, [' '.join(list_2)])

print('\nThe cosine similarity for the first list is {}.'.format(list_1_cos))
print('\nThe cosine similarity for the second list is {}.'.format(list_2_cos))

In [236]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def vect_cos(vect, test_list):

    """ Vectorise text and compute the cosine similarity """
    query_0 = vect.transform([' '.join(vect.get_feature_names())])
    query_1 = vect.transform(test_list)
    cos_sim = cosine_similarity(query_0.A, query_1.A)  # displays the resulting matrix
    return query_1, np.round(cos_sim.squeeze(), 3)

In [246]:
see = ['adventure', 'mystery', 'thriller', 'fantasy']
see_vect, see_cos = vect_cos(vectoriser3, [' '.join(see)])
print('\nThe cosine similarity for the first list is {}.'.format(see_cos))
# ได้แล้วโว้ยยยยยยย



The cosine similarity for the first list is 0.632.


In [233]:
see = []
for item in splittedgenres[0]:
    perrow = item.lower()
    see.append(perrow)
    

In [232]:
print(item.lower())

fantasy


In [234]:
print(see)

['adventure', 'animation', 'children', 'comedy', 'fantasy']


In [243]:
all_movies['genres'][8865]

'Comedy|Romance|SciFi'

In [ ]:
splitted genres
then item.lower()
then see_vect, see_cos = vect_cos(vectoriser3, [' '.join(see)])
ออกมาเป็นค่า cosine_sim ระหว่าง input genres กับ genres ของทีละบรรทัด
#wuuuuuhuuuuuwwww

In [192]:
flat_list

['Action', 'SciFi', 'Mystery', 'SciFi', 'Thriller', 'SciFi', 'IMAX']

In [168]:
products_dict['Action|SciFi|Mystery|SciFi|Thriller|SciFi|IMAX']

KeyError: 'Action|SciFi|Mystery|SciFi|Thriller|SciFi|IMAX'

In [ ]:
เอ๊า ใสได้เฉพาะอันที่มี แต่คิดให้อันที่ใกล้เคียงไม่เป็นเหรอ

In [172]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
documents=(all_movies["genres"])

tfidf_vectorizer=TfidfVectorizer()
tfidf_matrix=tfidf_vectorizer.fit_transform(documents)
#print tfidf_matrix.shape

cs=cosine_similarity(tfidf_matrix,tfidf_matrix)
print (cs)

[[1.         0.81357774 0.15276924 ... 0.         0.4210373  0.26758648]
 [0.81357774 1.         0.         ... 0.         0.         0.        ]
 [0.15276924 0.         1.         ... 0.         0.         0.57091541]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.4210373  0.         0.         ... 0.         1.         0.        ]
 [0.26758648 0.         0.57091541 ... 0.         0.         1.        ]]


In [173]:
cs.shape

(9742, 9742)

In [182]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
documents=(all_movies["genres"])

tfidf_vectorizer=TfidfVectorizer()
tfidf_matrix=tfidf_vectorizer.fit_transform(documents)
#print tfidf_matrix.shape

cs2=cosine_similarity(myfavgenres,tfidf_matrix)
print (cs2)

ValueError: could not convert string to float: 'Action|SciFi|Mystery|SciFi|Thriller|SciFi|IMAX'

In [174]:
flat_list

['Action', 'SciFi', 'Mystery', 'SciFi', 'Thriller', 'SciFi', 'IMAX']

In [180]:
myfavgenres = 'Action|SciFi|Mystery|SciFi|Thriller|SciFi|IMAX'

In [176]:
all_movies['genres']

0       Adventure|Animation|Children|Comedy|Fantasy
1                        Adventure|Children|Fantasy
2                                    Comedy|Romance
3                              Comedy|Drama|Romance
4                                            Comedy
                           ...                     
9737                Action|Animation|Comedy|Fantasy
9738                       Animation|Comedy|Fantasy
9739                                          Drama
9740                               Action|Animation
9741                                         Comedy
Name: genres, Length: 9742, dtype: object

In [242]:
all_movies['genres'][8865]

'Comedy|Romance|SciFi'